In [17]:
# ladowanie modulu od unittestow
%load_ext ipython_unittest
from collections import Counter
from utils import *
import requests

The ipython_unittest extension is already loaded. To reload it, use:
  %reload_ext ipython_unittest


In [36]:
%%unittest_main

class MergeTests(unittest.TestCase):
    @classmethod
    def setUpClass(cls):
        reqUsers = requests.get("https://jsonplaceholder.typicode.com/users")
        reqPosts = requests.get("https://jsonplaceholder.typicode.com/posts")
        if [reqUsers.status_code, reqPosts.status_code] != [200, 200]: 
            raise Exception("Error: couldn't retrieve data from the server") 
        cls.posts = reqPosts.json()
        cls.users = reqUsers.json()

    def testPostsNotNull(self):
        for i in self.posts:
            for j in i:
                self.assertIsNotNone(j)

    def testUsersNotNull(self):
        for i in self.users:
            for j in i:
                self.assertIsNotNone(j)

    def testMergedListHasAllFields(self):
        postKeys = len(self.posts[0].keys())
        userKeys = len(self.users[0].keys())
        mL = mergeLists(self.posts, self.users, "userId")
        self.assertEqual(len(mL[0].keys()), postKeys+userKeys-1)

    

Success

...
----------------------------------------------------------------------
Ran 3 tests in 0.193s

OK


<unittest.runner.TextTestResult run=3 errors=0 failures=0>